<center style="
               padding: 2rem 3rem;
               border: 3px solid #aae629;
               border-radius: 20px;
               ">
    <h1 style="color: #aae629;">ConvNeXt-tiny</h1>
    <h2 style="color: #aae629;">⚡️ Pytorch Lightning + timm 📷</h2>
    <a href="https://kaggle.com/shreydan" style="text-decoration:none;
                                                 padding: 1rem 2rem;
                                                 color: white;
                                                 background-color: #aae629;
                                                 border-radius: 30px;
                                                 font-size:1.25rem;
                                                 " >@shreydan</a>
</center>

# **Imports**
___

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.model_selection import train_test_split

In [2]:
import torch
import torch.nn as nn
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
import albumentations as A
from albumentations.pytorch import ToTensorV2
from PIL import Image

# **Setting up Timm**
___

In [3]:
import os
if not os.path.exists('/root/.cache/torch/hub/checkpoints/'):
    os.makedirs('/root/.cache/torch/hub/checkpoints/')
!cp '../input/convnext-tiny-1k-224-ema-weights/convnext_tiny_1k_224_ema.pth' '/root/.cache/torch/hub/checkpoints/convnext_tiny_1k_224_ema.pth'

In [4]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
from timm import create_model

# **Preparing Datasets**
___

In [5]:
train_images = list(Path('../input/petfinder-pawpularity-score/train').glob("*"))

# just noise
test_images = list(Path('../input/petfinder-pawpularity-score/test').glob("*"))

len(train_images)

9912

In [6]:
# checking image types
Counter([path.suffix for path in train_images])

Counter({'.jpg': 9912})

In [7]:
df = pd.read_csv('../input/petfinder-pawpularity-score/train.csv')
df = df.sample(frac=1).reset_index(drop=True) # shuffle

In [8]:
# since all images have .jpg extension
base_path = '../input/petfinder-pawpularity-score/train/'
get_path = lambda x: base_path + x + '.jpg'
df['path'] = df['Id'].apply(get_path)
df['norm'] = df['Pawpularity'] / 100.0
df.head(1)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,Pawpularity,path,norm
0,7c1ac48473361fd79fc10538202d7d3c,0,1,1,1,0,0,0,0,1,0,0,0,30,../input/petfinder-pawpularity-score/train/7c1...,0.3


In [9]:
test_df = pd.read_csv('../input/petfinder-pawpularity-score/test.csv')
base_path = '../input/petfinder-pawpularity-score/test/'
get_path = lambda x: base_path + x + '.jpg'
test_df['path'] = test_df['Id'].apply(get_path)
test_df.head(1)

,Id,Subject Focus,Eyes,Face,Near,Action,Accessory,Group,Collage,Human,Occlusion,Info,Blur,path
0,4128bae22183829d2b5fea10effdb0c3,1,0,1,0,0,1,1,0,0,1,0,1,../input/petfinder-pawpularity-score/test/4128...


In [10]:
train_df, val_df = train_test_split(df, test_size=0.2, shuffle=True, random_state=1357)
train_df.reset_index(drop=True, inplace=True)
val_df.reset_index(drop=True, inplace=True)

len(train_df), len(val_df)

(7929, 1983)

# **Albumentations**
___

In [11]:
train_augs = A.Compose(
    [
        A.Resize(height=320, width=320),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.RandomCrop(height=224, width=224),
        A.Normalize(mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225), 
                    always_apply=True
                   ),
        ToTensorV2(),
    ],
    p=1.0
)
val_augs = A.Compose(
    [
        A.Resize(height=320, width=320),
        A.CenterCrop(height=224, width=224),
        A.Normalize(mean=(0.485, 0.456, 0.406), 
                    std=(0.229, 0.224, 0.225),
                    always_apply=True
                   ),
        ToTensorV2()
    ],
    p=1.0
)

# **Custom Dataset**
___

In [12]:
class PawsDataset:
    def __init__(self, df, augs, is_test=False):
        self.paths = df['path'].values
        self.meta = df.iloc[:,1:13].values
        self.augs = augs
        self.is_test = is_test
        if not self.is_test:
            self.scores = df['norm'].values
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, idx):
        sample = self.paths[idx]
        sample = Image.open(sample).convert(mode='RGB')
        sample = np.array(sample)
        sample = self.augs(image=sample)['image']
        meta = torch.tensor(self.meta[idx,:], dtype=torch.float32)
        if not self.is_test:
            score = torch.tensor(self.scores[idx], dtype=torch.float32)
            return sample, meta, score
        
        return sample, meta
        

In [13]:
train_ds = PawsDataset(train_df, augs=train_augs)
val_ds = PawsDataset(val_df, augs=val_augs)
test_ds = PawsDataset(test_df, augs=val_augs, is_test=True)

# **ConvNeXt-Tiny**
___

## [Paper: A ConvNet for the 2020s](https://arxiv.org/abs/2201.03545)

## Reason for using this model:

Recently, Professor Jeremy Howard released a notebook: [The best vision models for fine-tuning](https://www.kaggle.com/code/jhoward/the-best-vision-models-for-fine-tuning) in which he says:

>The excellent showing of `convnext_tiny` matches my view that we should think of this as our default baseline for image recognition today. It's fast, accurate, and not too much of a memory hog. 

That's why I wanted to try it. I hadn't tried fast.ai before so chose to go with Pytorch Lightning 



In [14]:
m = create_model('convnext_tiny', pretrained=True, num_classes = 1)
m.head

Sequential(
  (global_pool): SelectAdaptivePool2d (pool_type=avg, flatten=Identity())
  (norm): LayerNorm2d((768,), eps=1e-06, elementwise_affine=True)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (drop): Dropout(p=0.0, inplace=False)
  (fc): Linear(in_features=768, out_features=1, bias=True)
)

# **Lightning Model Wrapper**
___

In [15]:
class PawsModel(pl.LightningModule):
    def __init__(self, model_name='convnext_tiny', dropout=0.1):
        super(PawsModel, self).__init__()
        self.model_name = model_name
        self.backbone = create_model(self.model_name, pretrained=True, num_classes = 0)
        self.drop = nn.Dropout(dropout)
        self.fc = nn.LazyLinear(1)
        self.mse = nn.MSELoss()
        
        self.test_preds = []
        
    def RMSE(self, preds, y):
        mse = self.mse(preds.view(-1), y.view(-1))
        return torch.sqrt(mse)
        
    def forward(self, sample):
        x,m = sample
        x = self.backbone(x)
        x = self.drop(x)
        cat = torch.cat([x,m],dim=1)
        logit = self.fc(cat)
        return logit
    
    def training_step(self, batch, batch_idx):
        
        *sample,y = batch
        
        preds = self(sample)
        
        loss = self.RMSE(preds, y)
        self.log('train_loss', loss.item(), on_epoch=True, prog_bar=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        
        *sample,y = batch
        
        preds = self(sample)
        
        loss = self.RMSE(preds, y)
        self.log('val_loss', loss.item(), on_epoch=True, prog_bar=True)
        
        
    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-5)
        return optimizer
    
    def test_step(self, batch, batch_idx):
        sample = batch
        preds = 100 * self(sample)
        self.test_preds.append(preds.detach().cpu())
        
    def get_predictions(self):
        return torch.cat(self.test_preds).numpy()

# **DataLoaders**
___

In [16]:
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=32, num_workers=2, shuffle=True)
val_dl = torch.utils.data.DataLoader(val_ds, batch_size=32, num_workers=2, shuffle=False)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=1, num_workers=1, shuffle=False)
len(train_dl), len(val_dl), len(test_dl)

(248, 62, 8)

# **Training**
___

In [17]:
model=PawsModel()
trainer = pl.Trainer(accelerator='gpu', 
                     max_epochs=6, 
                     callbacks=[
                         EarlyStopping(monitor="val_loss", 
                                       mode="min",
                                       patience=2,
                                      )
                        ]
                    )
trainer.fit(model, train_dl, val_dl)

/opt/conda/lib/python3.7/site-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/utilities/model_summary.py:410: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  "A layer with UninitializedParameter was found. "


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

# **Testing**
___

In [18]:
trainer.test(model,test_dl)

Testing: 0it [00:00, ?it/s]

[{}]

In [19]:
p = model.get_predictions().reshape(-1)
p

array([43.25495 , 37.177082, 42.498035, 35.276085, 34.25432 , 31.665758,
       40.561943, 33.115284], dtype=float32)

In [20]:
submission_df = pd.DataFrame({'Id': test_df['Id'].values, 'Pawpularity': p})
submission_df.to_csv('submission.csv',index=False)

In [21]:
submission_df

,Id,Pawpularity
0,4128bae22183829d2b5fea10effdb0c3,43.254951
1,43a2262d7738e3d420d453815151079e,37.177082
2,4e429cead1848a298432a0acad014c9d,42.498035
3,80bc3ccafcc51b66303c2c263aa38486,35.276085
4,8f49844c382931444e68dffbe20228f4,34.254318
5,b03f7041962238a7c9d6537e22f9b017,31.665758
6,c978013571258ed6d4637f6e8cc9d6a3,40.561943
7,e0de453c1bffc20c22b072b34b54e50f,33.115284


## Additional Reference Notebooks:

- [Petfinder Pawpularity EDA & fastai starter 🐱🐶](https://www.kaggle.com/code/tanlikesmath/petfinder-pawpularity-eda-fastai-starter/notebook) : gave me reference for learning rate + this was mentioned in Prof. Jeremy Howard's notebook as well

- [[Pytorch + W&B] Pawpularity Training 🔥](https://www.kaggle.com/code/debarshichanda/pytorch-w-b-pawpularity-training) : gave me reference for how to combine feature extractions and meta features, and loss functions


### **✨️ Thank You for taking the time to check out my notebook ✨️**


___

<center>
    <img src="https://img.shields.io/badge/Upvote-If%20you%20like%20my%20work-07b3c8?style=for-the-badge&logo=kaggle"/>
</center>